In [ ]:
%reset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv('../data/archive/fashion-mnist_train.csv')
display(df.shape)

# training test split
x_train, x_test, t_train, t_test =\
train_test_split(df.drop(columns=['label']), pd.get_dummies(df['label']), 
                 test_size=0.2, random_state=0)

# Normaliztion
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)

# Mutinomial Classification

In [ ]:
x_dim = x_train.shape[1]
t_dim = t_train.shape[1]

# placeholder
X = tf.placeholder(shape=[None, x_dim], dtype=tf.float32)
T = tf.placeholder(shape=[None, t_dim], dtype=tf.float32)

# W & b
W = tf.get_variable('W', shape=[x_dim, t_dim],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random.normal([t_dim]))

# Hypothesis
logit = tf.matmul(X, W) + b
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# learning
for step in range(1000):
    _,loss_val = sess.run([train,loss], feed_dict={X:x_train_norm, T:t_train})
    
    if step % 100 == 0:
        print(f'loss: {loss_val}')

In [ ]:
# accuracy
predict = np.argmax(sess.run(H, feed_dict = {X: x_test_norm}), axis=1)
acc_list = np.equal(predict, np.argmax(t_test.values, axis=1))
print('accuracy : {}'.format(acc_list.sum()/len(acc_list)))

# Deep Nueral Network

In [ ]:
%reset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv('../data/archive/fashion-mnist_train.csv')
display(df.shape)

# training test split
x_train, x_test, t_train, t_test =\
train_test_split(df.drop(columns=['label']), pd.get_dummies(df['label']), 
                 test_size=0.2, random_state=0)

# Normaliztion
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)

In [ ]:
x_dim = x_train.shape[1]
t_dim = t_train.shape[1]

# placeholder
X = tf.placeholder(shape=[None, x_dim], dtype=tf.float32)
T = tf.placeholder(shape=[None, t_dim], dtype=tf.float32)

# W & b
W2 = tf.get_variable('W2', shape=[x_dim, 64],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random.normal([64]))
layer2 = tf.nn.relu(tf.matmul(X, W2) + b2)

W3 = tf.get_variable('W3', shape=[64, 32], 
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random.normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

W4 = tf.get_variable('W4', shape=[32, 16], 
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random.normal([16]))
layer4 = tf.nn.relu(tf.matmul(layer3, W4) + b4)

W5 = tf.get_variable('W5', shape=[16, t_dim], 
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random.normal([t_dim]))

# hypothesis
logit = tf.matmul(layer4, W5) +b5
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# learning
for step in range(1000):
    _,loss_val = sess.run([train,loss], feed_dict={X:x_train_norm, T:t_train})
    
    if step % 100 == 0:
        print(f'loss: {loss_val}')

In [ ]:
# accuracy
predict = np.argmax(sess.run(H, feed_dict = {X: x_test_norm}), axis=1)
acc_list = np.equal(predict, np.argmax(t_test.values, axis=1))
print('accuracy : {}'.format(acc_list.sum()/len(acc_list)))

# Convolution Neural Network

In [ ]:
%reset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv('../data/archive/fashion-mnist_train.csv')

# training test split
x_train, x_test, t_train, t_test =\
train_test_split(df.drop(columns=['label']).values, 
                 pd.get_dummies(df['label']).values, 
                 test_size=0.2, random_state=0)

# Normaliztion
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)

In [2]:
# 형태 변환
x_train_norm = x_train_norm.reshape((x_train_norm.shape[0], ) + (28, 28, 1))
x_test_norm = x_test_norm.reshape((x_test_norm.shape[0], ) + (28, 28, 1))

In [3]:
# placeholder
X = tf.placeholder(shape=[None, 28, 28 ,1], dtype=tf.float32)
T = tf.placeholder(shape=[None, 10], dtype=tf.float32)

# filter
weight = tf.get_variable('filter', shape=[3, 3, 1, 1],
                         initializer=tf.contrib.layers.xavier_initializer())

# convolution
conv2d = tf.nn.conv2d(X, weight, 
                      strides=[1,1,1,1], padding='VALID')

# relu
relu = tf.nn.relu(conv2d)

# pooling
pooling = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], padding='VALID')

# filter 2
weight2 = tf.get_variable('filter2', shape=[3, 3, 1, 1],
                         initializer=tf.contrib.layers.xavier_initializer())

# convolution 2
conv2d2 = tf.nn.conv2d(pooling, weight, 
                      strides=[1,1,1,1], padding='VALID')

# relu 2
relu2 = tf.nn.relu(conv2d2)

# pooling 2
pooling2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], padding='VALID')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
# flatten
flatten = tf.layers.Flatten()(pooling2)

# W & b
W = tf.get_variable('W', shape=[25, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random.normal([10]))

# hypothesis
logit = tf.matmul(flatten, W) +b
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
# learning
for step in range(10000):
    _,loss_val = sess.run([train,loss], feed_dict={X:x_train_norm, T:t_train})
    
    if step % 1000 == 0:
        print(f'loss: {loss_val}')

loss: 0.7932090759277344
loss: 0.7379981875419617
loss: 0.7116959691047668
loss: 0.7014251351356506
loss: 0.6956386566162109
loss: 0.6907168030738831
loss: 0.6872500777244568
loss: 0.6840869784355164
loss: 0.6808200478553772
loss: 0.6778950095176697


In [8]:
# accuracy
predict = np.argmax(sess.run(H, feed_dict = {X: x_test_norm}), axis=1)
acc_list = np.equal(predict, np.argmax(t_test, axis=1))
print('accuracy : {}'.format(acc_list.sum()/len(acc_list)))

accuracy : 0.7579166666666667
